# **Instalar bibliotecas necesarias**

In [3]:
%%capture
!pip install langchain-community==0.2.1
!pip install pypdf==4.2.0
!pip install PyMuPDF==1.24.5
!pip install unstructured==0.14.8
!pip install markdown==3.6
!pip install jq==1.7.0
!pip install pandas==2.2.2
!pip install docx2txt==0.8
!pip install requests==2.32.3
!pip install beautifulsoup4==4.12.3
!pip install nltk==3.8.0
!pip install pypdfium2
!pip install arxiv
# Dependencias extra para unstructured que a veces fallan en colab
!pip install python-magic
!apt-get install poppler-utils tesseract-ocr


# **Importar bibliotecas**

In [1]:
# Importar bibliotecas
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

import os
import gradio as gr

# Cargar documentos

In [2]:
def cargar_documentos(ruta_archivo, tipo_documento):
    if tipo_documento == "pdf":
        loader = PyPDFLoader(ruta_archivo)
    elif tipo_documento == "docx":
        loader = Docx2txtLoader(ruta_archivo)
    elif tipo_documento == "txt":
        loader = TextLoader(ruta_archivo)
    else:
        raise ValueError("Tipo de documento no soportado.")
    return loader.load()


# Dividir documentos en fragmentos

In [3]:
def dividir_documentos(documentos):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documentos)

# Crear índice de búsqueda

In [4]:
def crear_indice(documentos):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    return FAISS.from_documents(documentos, embeddings)

# Función para la respuesta

In [5]:


def responder_pregunta(contexto, pregunta):
    # Usar un modelo de question-answering más robusto
    qa_pipeline = pipeline(
        "question-answering",
        model="deepset/roberta-base-squad2",
        tokenizer="deepset/roberta-base-squad2"
    )

    # Obtener la respuesta
    respuesta = qa_pipeline(question=pregunta, context=contexto)

    # Verificar si se obtuvo una respuesta
    if respuesta and 'answer' in respuesta:
        return respuesta["answer"]
    else:
        return "No se encontró una respuesta adecuada en el documento."

# Función para el chatbot

In [10]:
# Cargar el documento
ruta_archivo = "/content/Articulo.pdf"
documentos = cargar_documentos(ruta_archivo, "pdf")

def chatbot_soporte_tecnico(ruta_documento, pregunta):
    # Cargar y dividir el documento
    documentos = cargar_documentos(ruta_documento, "pdf")
    fragmentos = dividir_documentos(documentos)

    # Crear índice y buscar fragmentos relevantes
    indice = crear_indice(fragmentos)
    resultados = indice.similarity_search(pregunta)

    # Obtener el fragmento más relevante
    contexto = resultados[1].page_content

    # Responder la pregunta
    respuesta = responder_pregunta(contexto, pregunta)
    return respuesta


# **Ejemplo de uso**

In [11]:
# Ejemplo de uso
respuesta = chatbot_soporte_tecnico(ruta_archivo, "¿Cuál es la metodología?")
print(respuesta)

Device set to use cpu


cuatro etapas


# **Cambio de modelo de generación de texto para obtener respuestas más extensas:**

Modelo de Generación de Texto:

Se cambió a google/flan-t5-base, un modelo de generación de texto que puede producir respuestas más largas y detalladas.
Se configuró max_length=512 para permitir respuestas más largas.
Se usó do_sample=True y temperature=0.7 para generar respuestas más variadas y naturales.


Tamaño de los Fragmentos:

Se aumentó chunk_size a 2000 y chunk_overlap a 400 para capturar más contexto en cada fragmento.


Combinar Múltiples Fragmentos:

Se obtienen los 3 fragmento



In [12]:
from transformers import pipeline

def responder_pregunta(contexto, pregunta):
    # Usar un modelo de generación de texto para respuestas más largas
    qa_pipeline = pipeline(
        "text2text-generation",
        model="google/flan-t5-base",
        tokenizer="google/flan-t5-base",
        max_length=512,
        do_sample=True,
        temperature=0.7
    )

    # Formatear la entrada para el modelo
    entrada = f"documento: {contexto}\npregunta: {pregunta}\nrespuesta:"
    respuesta = qa_pipeline(entrada)

    # Verificar si se obtuvo una respuesta
    if respuesta and len(respuesta) > 0 and 'generated_text' in respuesta[0]:
        return respuesta[0]['generated_text']
    else:
        return "No se encontró una respuesta adecuada en el documento."

def dividir_documentos(documentos):
    # Aumentar el tamaño de los fragmentos para capturar más contexto
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
    return text_splitter.split_documents(documentos)

def chatbot_soporte_tecnico(ruta_documento, pregunta):
    # Cargar y dividir el documento
    documentos = cargar_documentos(ruta_documento, "pdf")
    fragmentos = dividir_documentos(documentos)

    # Crear índice y buscar fragmentos relevantes
    indice = crear_indice(fragmentos)
    resultados = indice.similarity_search(pregunta, k=3)  # Obtener los 3 fragmentos más relevantes

    # Combinar los fragmentos para dar más contexto
    contexto_combinado = "\n\n".join([resultado.page_content for resultado in resultados])

    # Responder la pregunta
    respuesta = responder_pregunta(contexto_combinado, pregunta)
    return respuesta


# **Ejemplos de uso**

In [13]:
# Ejemplo de uso
ruta_archivo = "/content/Articulo.pdf"
respuesta = chatbot_soporte_tecnico(ruta_archivo, "¿De qué trata el artículo?")
print("Respuesta:", respuesta)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (3530 > 512). Running this sequence through the model will result in indexing errors


Respuesta: propone la identificación de las condiciones de adquisición de imágenes en zonas urbanas para detectar residuos sólidos mediante visión artificial


In [17]:
# Ejemplo de uso 2
respuesta = chatbot_soporte_tecnico(ruta_archivo, "¿que son los parámetros de caracterización?")
print("Respuesta:", respuesta)

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (2814 > 512). Running this sequence through the model will result in indexing errors


Respuesta: a) altura de la cámara, el ángulo de inclinación y el nivel de luminosidad en la zona donde se tomaron las imágenes [11]. a) Altura de la cámara: Las imágenes se tomaron en las zonas seleccionadas a alturas de 0.5 m y 1.5 m con respecto a la superficie del canal. Estas posiciones se definieron mediante el análisis de factores como, la perspectiva, una menor variabilidad en el tamao de los residuos observados, ajustando los ángulos para la obtención de una visión adecuada de los residuos, teniendo también las propias restricciones del lente de la cámara que determinan el ángulo de visión en la captura de las imágenes [13]. c) Luminosidad: Para evaluar el nivel de luminosidad al momento de realizar la adquisición de imágenes se empleó dos dispositivos de captura de 12 MP f/1.6, OIS y 64 MP OIS f/1.9. b) ngulo de inclinación: la inclinación del ángulo del dispositivo de captura de imágenes fueron de 20° y 60°. Estos ángulos establecieron teniendo también las caractersticas prop